# Housing Sale Prediction

In [90]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

## 1. Data Import and Basic Preprocessing

### 1.1 Data Loading and Check for NAN

In [91]:
train_df = pd.read_csv('../data/housing/train.csv', keep_default_na=False)
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000


In [92]:
train_df.dtypes.unique()

array([dtype('int64'), dtype('O')], dtype=object)

In [93]:
train_df.isna().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [94]:
test_df = pd.read_csv('../data/housing/test.csv', keep_default_na=False)
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80,11622,Pave,NA,Reg,Lvl,AllPub,...,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal
1,1462,20,RL,81,14267,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74,13830,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,MnPrv,NA,0,3,2010,WD,Normal
3,1464,60,RL,78,9978,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,NA,0,6,2010,WD,Normal
4,1465,120,RL,43,5005,Pave,NA,IR1,HLS,AllPub,...,144,0,NA,NA,NA,0,1,2010,WD,Normal


In [95]:
test_df.isna().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 80, dtype: int64

In [96]:
test_df.dtypes

Id                int64
MSSubClass        int64
MSZoning         object
LotFrontage      object
LotArea           int64
                  ...  
MiscVal           int64
MoSold            int64
YrSold            int64
SaleType         object
SaleCondition    object
Length: 80, dtype: object

## 2. Initial Model Training

Intend to use a simple pipeline with numerical and categorical encoder to get initial MAE, then look to improve model results.

In [97]:
def y_and_train_test_split(df: pd.DataFrame, y_column: str, train_size: float):
    """_summary_

    Args:
        df (pd.DataFrame): _description_
        train_size (float): _description_
    """
    test_size = 1.0 - train_size
    X_final = df.drop(y_column, axis=1)
    y = df[y_column]
    return train_test_split(X_final, y,
                            train_size=train_size, test_size=test_size,
                            random_state=0)

### 2.1 Create test_train split

In [98]:
y_column = 'SalePrice'
X_train, X_valid, y_train, y_valid = y_and_train_test_split(train_df, y_column, 0.8)

In [99]:
X_train.shape

(1168, 80)

In [100]:
X_valid.shape

(292, 80)

### 2.2 Generating Categorical and Numerical col_names

These will be passed to the Pipeline encoders.

In [101]:
categorical_cols = [cname for cname in X_train.columns
                    if X_train[cname].nunique() < 10 and
                    X_train[cname].dtype == "object"]
categorical_cols

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [102]:
numerical_cols = [cname for cname in X_train.columns
                  if X_train[cname].dtype in ['int64', 'float64']]
numerical_cols

['Id',
 'MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [103]:
train_df[numerical_cols].dtypes

Id               int64
MSSubClass       int64
LotArea          int64
OverallQual      int64
OverallCond      int64
YearBuilt        int64
YearRemodAdd     int64
BsmtFinSF1       int64
BsmtFinSF2       int64
BsmtUnfSF        int64
TotalBsmtSF      int64
1stFlrSF         int64
2ndFlrSF         int64
LowQualFinSF     int64
GrLivArea        int64
BsmtFullBath     int64
BsmtHalfBath     int64
FullBath         int64
HalfBath         int64
BedroomAbvGr     int64
KitchenAbvGr     int64
TotRmsAbvGrd     int64
Fireplaces       int64
GarageCars       int64
GarageArea       int64
WoodDeckSF       int64
OpenPorchSF      int64
EnclosedPorch    int64
3SsnPorch        int64
ScreenPorch      int64
PoolArea         int64
MiscVal          int64
MoSold           int64
YrSold           int64
dtype: object

In [ ]:
test_df[numerical_cols].dtypes

In [81]:
full_cols = categorical_cols + numerical_cols
X_train = X_train[full_cols].copy()
X_valid = X_valid[full_cols].copy()
X_test = test_df[full_cols].copy()

### 2.3 Preprocess and Train Model

Define the numerical and categorical encoders.

In [82]:
numerical_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

In [83]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [84]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
clf = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ]
)

In [85]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivArea', 'BsmtFullBath',
                                                   'BsmtHalfBath', 'F...
                                                   'LotConfig', 'LandSlope',
                                                   'Condition1', 'Condition2',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'MasVnrType', 'ExterQual',
                                                   'ExterCond', 'Foundation',
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...])])),
                ('model', RandomForestRegressor(random_state=0))])

In [86]:
preds = clf.predict(X_valid)

In [87]:
mea = mean_absolute_error(y_valid, preds)
mea

17797.981061643837

In [88]:
X_test[numerical_cols].dtypes

Id                int64
MSSubClass        int64
LotFrontage       int64
LotArea           int64
OverallQual       int64
OverallCond       int64
YearBuilt         int64
YearRemodAdd      int64
MasVnrArea        int64
BsmtFinSF1       object
BsmtFinSF2       object
BsmtUnfSF        object
TotalBsmtSF      object
1stFlrSF          int64
2ndFlrSF          int64
LowQualFinSF      int64
GrLivArea         int64
BsmtFullBath     object
BsmtHalfBath     object
FullBath          int64
HalfBath          int64
BedroomAbvGr      int64
KitchenAbvGr      int64
TotRmsAbvGrd      int64
Fireplaces        int64
GarageYrBlt       int64
GarageCars       object
GarageArea       object
WoodDeckSF        int64
OpenPorchSF       int64
EnclosedPorch     int64
3SsnPorch         int64
ScreenPorch       int64
PoolArea          int64
MiscVal           int64
MoSold            int64
YrSold            int64
dtype: object

In [89]:
preds_test = clf.predict(X_test)
preds_test

ValueError: could not convert string to float: 'NA'

### 1.3 Ordinal Encoding

Identified the below cols as ordinal values.

In [21]:
ordinal_cols = [
    'LandSlope',
    'OverallQual',
    'OverallCond',
    'ExterQual',
    'ExterCond',
    'BsmtQual',
    'BsmtCond',
    'BsmtExposure',
    'BsmtFinType1',
    'BsmtFinType2',
    'HeatingQC',
    'KitchenQual',
    'FireplaceQu',
    'GarageFinish',
    'GarageQual',
    'GarageCond',
    'PoolQC',
    'Fence'
]

In [22]:
train_df['LandSlope'].unique()

array(['Gtl', 'Mod', 'Sev'], dtype=object)

In [23]:
full_categories = [np.unique(np.append(train_df[col], test_df[col])) for col in ordinal_cols]
full_categories

[array(['Gtl', 'Mod', 'Sev'], dtype=object),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array(['Ex', 'Fa', 'Gd', 'TA'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'Po', 'TA'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'NA', 'TA'], dtype=object),
 array(['Fa', 'Gd', 'NA', 'Po', 'TA'], dtype=object),
 array(['Av', 'Gd', 'Mn', 'NA', 'No'], dtype=object),
 array(['ALQ', 'BLQ', 'GLQ', 'LwQ', 'NA', 'Rec', 'Unf'], dtype=object),
 array(['ALQ', 'BLQ', 'GLQ', 'LwQ', 'NA', 'Rec', 'Unf'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'Po', 'TA'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'NA', 'TA'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'NA', 'Po', 'TA'], dtype=object),
 array(['Fin', 'NA', 'RFn', 'Unf'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'NA', 'Po', 'TA'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'NA', 'Po', 'TA'], dtype=object),
 array(['Ex', 'Fa', 'Gd', 'NA'], dtype=object),
 array(['GdPrv', 'GdWo', 'MnPrv', 'MnWw', 'NA'], dtype=object)]

In [24]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(categories=full_categories)

In [25]:
train_df[ordinal_cols] = ordinal_encoder.fit_transform(train_df[ordinal_cols])

In [26]:
test_df[ordinal_cols] = ordinal_encoder.transform(test_df[ordinal_cols])

### 1.4 Binary Encoding

Only need to binary encode central air.

In [27]:
train_df['CentralAir'] = train_df['CentralAir'].replace({'Yes': 1, "No": 0})
test_df['CentralAir'] = test_df['CentralAir'].replace({'Yes': 1, "No": 0})

### 1.5 One Hot Encoder Encoding

In [28]:
columns_to_ignore = np.append(ordinal_cols, 'CentralAir')

In [29]:
categorical_mask = (train_df.dtypes == object)
categorical_cols = train_df.columns[categorical_mask]
categorical_cols = [col for col in categorical_cols if col not in columns_to_ignore]
print(f"num categorical cols {len(categorical_cols)}")

num categorical cols 26


In [31]:
train_df.drop('Id', axis=1, inplace=True)
train_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,3.0,4.0,NA,0,2,2008,WD,Normal,208500
1,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,3.0,4.0,NA,0,5,2007,WD,Normal,181500
2,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,3.0,4.0,NA,0,9,2008,WD,Normal,223500
3,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,3.0,4.0,NA,0,2,2006,WD,Abnorml,140000
4,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,0,3.0,4.0,NA,0,12,2008,WD,Normal,250000


In [32]:
one_hot_training_values = (train_df[categorical_cols]
                           .apply(lambda x : x.unique())
                           .apply(list))
one_hot_training_values

MSZoning                                 [RL, RM, C (all), FV, RH]
Street                                                [Pave, Grvl]
Alley                                             [NA, Grvl, Pave]
LotShape                                      [Reg, IR1, IR2, IR3]
LandContour                                   [Lvl, Bnk, Low, HLS]
Utilities                                         [AllPub, NoSeWa]
LotConfig                      [Inside, FR2, Corner, CulDSac, FR3]
Neighborhood     [CollgCr, Veenker, Crawfor, NoRidge, Mitchel, ...
Condition1       [Norm, Feedr, PosN, Artery, RRAe, RRNn, RRAn, ...
Condition2       [Norm, Artery, RRNn, Feedr, PosN, PosA, RRAn, ...
BldgType                     [1Fam, 2fmCon, Duplex, TwnhsE, Twnhs]
HouseStyle       [2Story, 1Story, 1.5Fin, 1.5Unf, SFoyer, SLvl,...
RoofStyle               [Gable, Hip, Gambrel, Mansard, Flat, Shed]
RoofMatl         [CompShg, WdShngl, Metal, WdShake, Membran, Ta...
Exterior1st      [VinylSd, MetalSd, Wd Sdng, HdBoard, BrkFace,

Review of the above indicates that not all of these need to be one_hot encoded. Some contain largely numerical values.